Title and intro text here

In [ ]:
%matplotlib inline

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np


Import the original time series and histogram it

In [ ]:
all_data = np.loadtxt("double_well.asc")
all_histogram,bin_edges = np.histogram(all_data, 235, [-2.35,2.35], density=True)
time = np.arange(len(all_data))

Here's the time series 

In [ ]:
plt.plot(time, all_data, "-k", linewidth=0.1)
plt.figtext(0.5, 0.0, "Time")

plt.show()


Here's the histogram

In [ ]:
plt.plot(bin_edges[:-1], all_histogram, "-k")
plt.figtext(0.5, 0.0, "Value")
plt.figtext(-0.2, 0.5, "Prob Density")
plt.show()


In [ ]:
def downsample_both(downsample_factor):
    reshaped = np.reshape(all_data, (len(all_data)/downsample_factor,
                                    downsample_factor))
    time_reshaped = np.reshape(time, (len(all_data)/downsample_factor,
                                    downsample_factor))
    downsampled = reshaped[:,0]
    time_downsampled = time_reshaped[:,0]
    down_histogram,bin_edges = np.histogram(downsampled, 235, [-2.35,2.35], density=True)
    plt.plot(time_downsampled, downsampled, "-r")

    plt.figtext(0.5, 0.0, "Time")
    plt.figtext(-0.1, 0.5, "Value")
    plt.show()
    plt.plot(bin_edges[:-1], down_histogram, "-k")
    plt.figtext(0.5, 0.0, "Value")
    plt.figtext(-0.1, 0.5, "Prob Density")
    plt.show()
    

In [ ]:
downscale_options = (1,100,1000,2500, 5000,10000, 50000, 100000)
interact(downsample_both, downsample_factor=widgets.Dropdown(options=downscale_options, description="Downsample Factor" ) );

In [ ]:
def average_both(average_factor):
    reshaped = np.reshape(all_data, (len(all_data)/average_factor,
                                    average_factor))
    time_reshaped = np.reshape(time, (len(all_data)/average_factor,
                                    average_factor))
    averaged = np.add.reduce(reshaped, 1) / average_factor
    ave_histogram,bin_edges = np.histogram(averaged, 235, [-2.35,2.35], density=True)
    time_downsampled = time_reshaped[:,0]
    plt.plot(time_downsampled, averaged, "-r")

    plt.figtext(0.5, 0.0, "Time")
    plt.figtext(-0.1, 0.5, "Value")
    plt.show()
    plt.plot(bin_edges[:-1], all_histogram, "-k", bin_edges[:-1], ave_histogram, "-r")
    plt.figtext(0.5, 0.0, "Value")
    plt.figtext(-0.1, 0.5, "Prob Density")
    plt.show()

In [ ]:
average_options = (1,100,1000,2500, 5000,10000, 50000, 100000)
interact(average_both, average_factor=widgets.Dropdown(options=downscale_options, description="Downsample Factor" ) );
